In [1]:
import pandas as pd
import numpy as np

In [2]:
all_data = pd.read_pickle("../data.pkl")
all_data_33 = all_data[all_data.date_block_num == 34]
all_data = all_data[all_data.date_block_num < 34]
test_df  = pd.read_csv("../test.csv.gz")

In [3]:
from sklearn.model_selection import train_test_split

target = all_data.item_cnt_month.values
target[target>20] = 20

predictors = ['item_cnt_month_lag_1',         # "target_lag_1"
              'date_item_avg_item_cnt_lag_1', # "item_target_enc_lag_1",
              'date_shop_avg_item_cnt_lag_1', # "shop_target_enc_lag_1",
                                              # "item_target_enc_ave",  "shop_target_enc_ave",
                                              # "item_cat_target_enc_lag_1", 
              'date_shop_cat_avg_item_cnt_lag_1',  # "shop_item_cat_target_enc_lag_1",
              'date_shop_type_avg_item_cnt_lag_1', # "shop_super_cat_target_enc_lag_1",
              'date_type_avg_item_cnt_lag_1'  # "super_cat_target_enc_lag_1"
             ]

X_train, X_test, y_train, y_test = \
    train_test_split(all_data[predictors], target, test_size=0.2, random_state=123)

In [4]:
all_data[predictors].to_csv("ref.csv")

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators=64, max_features=3, n_jobs=6) #len(predictors)/2
model2.fit(X_train, y_train)
ypred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred))
print("RMSE: %f" % (rmse))

RMSE: 0.909246


In [6]:
from sklearn.ensemble import RandomForestRegressor
full_model = RandomForestRegressor(n_estimators=64, max_features=3, n_jobs=6) #len(predictors)/2
full_model.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=6,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [7]:
%store -r __prepare_submission

In [8]:
__prepare_submission

In [9]:
aliases = \
{"item_cnt_month_lag_1":          "item_cnt_month_lag_1",
 "date_item_avg_item_cnt_lag_1":  "date_item_avg_item_cnt_lag_1",
 "date_shop_avg_item_cnt_lag_1":  "date_shop_avg_item_cnt_lag_1",
 "date_shop_cat_avg_item_cnt_lag_1": "date_shop_cat_avg_item_cnt_lag_1",
 "date_shop_type_avg_item_cnt_lag_1":"date_shop_type_avg_item_cnt_lag_1",
 "date_type_avg_item_cnt_lag_1":  "date_type_avg_item_cnt_lag_1"
}

prepare_submission(full_model, predictors, aliases, "comparison_rf.csv")

In [10]:
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(X_test.columns, full_model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                                   score
item_cnt_month_lag_1               0.35118
date_item_avg_item_cnt_lag_1       0.260171
date_shop_cat_avg_item_cnt_lag_1   0.146621
date_shop_type_avg_item_cnt_lag_1  0.084455
date_shop_avg_item_cnt_lag_1       0.0837268
date_type_avg_item_cnt_lag_1       0.0738454


This is bullshit! Using less predictors from his cleaner data I got 5% better results on the leaderboard.